# Capítulo 10 
## _Agregação de dados e oprerações em grupos_

- Funcionamento de groupby 

In [70]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [71]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-1.333830,-0.205268
1,a,two,0.613694,-0.399197
2,b,one,0.112896,0.106394
3,b,two,0.712401,0.939147
4,a,one,-0.975835,1.007969


In [72]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [73]:
grouped.mean()

key1
a   -0.565324
b    0.412649
Name: data1, dtype: float64

In [74]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -1.154832
      two     0.613694
b     one     0.112896
      two     0.712401
Name: data1, dtype: float64

In [75]:
means.unstack()

key2,one,two
key1,,
a,-1.154832,0.613694
b,0.112896,0.712401


In [76]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.613694
            2006    0.112896
Ohio        2005   -0.310715
            2006   -0.975835
Name: data1, dtype: float64

In [77]:
df.groupby("key1").mean(numeric_only=True)

,data1,data2
key1,,
a,-0.565324,0.134501
b,0.412649,0.522770


In [78]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
one,-0.732256,0.303031
two,0.663048,0.269975


In [79]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one  -1.154832  0.401350
     two   0.613694 -0.399197
b    one   0.112896  0.106394
     two   0.712401  0.939147

In [80]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

- Iterando por grupos

In [81]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.333830 -0.205268
1    a  two  0.613694 -0.399197
4    a  one -0.975835  1.007969
b
  key1 key2     data1     data2
2    b  one  0.112896  0.106394
3    b  two  0.712401  0.939147


In [82]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.333830 -0.205268
4    a  one -0.975835  1.007969
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.613694 -0.399197
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.112896  0.106394
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.712401  0.939147


In [83]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.112896,0.106394
3,b,two,0.712401,0.939147


In [84]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [85]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in  grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.333830 -0.205268
1  0.613694 -0.399197
2  0.112896  0.106394
3  0.712401  0.939147
4 -0.975835  1.007969
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


- Selecionando uma coluna ou um subconjunto de colunas

In [86]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.401350
     two  -0.399197
b    one   0.106394
     two   0.939147

In [87]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one     0.401350
      two    -0.399197
b     one     0.106394
      two     0.939147
Name: data2, dtype: float64

- Agrupando com dicionários e Series

In [88]:
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-0.249200,0.565862,-1.010489,-1.355514,0.683414
Steve,1.515320,-1.009091,-0.300393,-0.141629,0.137657
Wes,1.562376,NaN,NaN,0.816511,-1.644632
Jim,0.540951,-0.269233,-0.183994,-1.973546,-0.951610
Travis,1.708480,-0.883692,2.988399,-1.371197,-1.246509


In [89]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-2.366003,1.000077
Steve,-0.442022,0.643886
Wes,0.816511,-0.082256
Jim,-2.157540,-0.679892
Travis,1.617202,-0.421721


In [90]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [91]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


- Agrupando com funções

In [92]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.854127,0.296629,-1.194483,-2.512549,-1.912827
5,1.515320,-1.009091,-0.300393,-0.141629,0.137657
6,1.708480,-0.883692,2.988399,-1.371197,-1.246509


In [93]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.249200  0.565862 -1.010489 -1.355514 -1.644632
  two  0.540951 -0.269233 -0.183994 -1.973546 -0.951610
5 one  1.515320 -1.009091 -0.300393 -0.141629  0.137657
6 two  1.708480 -0.883692  2.988399 -1.371197 -1.246509

- Agrupando por níveis de índices

In [94]:
column = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=column)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.593255 -0.198269 -0.689731 -2.765096 -0.681548
1      1.648707 -1.498162  0.219531 -0.738359  0.150221
2     -0.783731 -0.794861  0.298668 -2.588787 -0.846018
3      0.692111  2.202353  1.802420 -1.995469  1.265411

In [95]:
hier_df.groupby(level=['cty'], axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## _Agregação de dados_

In [96]:
df
grouped = df.groupby('key1')
grouped['data1'].nsmallest(2)

key1   
a     0   -1.333830
      4   -0.975835
b     2    0.112896
      3    0.712401
Name: data1, dtype: float64

In [97]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
#grouped.agg(peak_to_peak)


- Aplicação de função nas colunas e aplicações de vároas funções

In [98]:
tips = pd.read_csv('exemplos/tips.csv')

#acrescenta a porcentagem de gorjerta sobre o total de conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [99]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [100]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [101]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [102]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [103]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [104]:
result['total_bill']

count       mean    max
day  smoker                         
Fri  No          4  18.420000  22.75
     Yes        15  16.813333  40.17
Sat  No         45  19.661778  48.33
     Yes        42  21.276667  50.81
Sun  No         57  20.506667  48.17
     Yes        19  24.120000  45.35
Thur No         45  17.113111  41.19
     Yes        17  19.190588  43.11

In [105]:
ftuplas = [('Durchshnitt', 'mean'), ('Abweichug', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuplas)

tip_pct            total_bill            
            Durchshnitt Abweichug Durchshnitt   Abweichug
day  smoker                                              
Fri  No        0.151650  0.000791   18.420000   25.596333
     Yes       0.174783  0.002631   16.813333   82.562438
Sat  No        0.158048  0.001581   19.661778   79.908965
     Yes       0.147906  0.003767   21.276667  101.387535
Sun  No        0.160113  0.001793   20.506667   66.099980
     Yes       0.187250  0.023757   24.120000  109.046044
Thur No        0.160298  0.001503   17.113111   59.625081
     Yes       0.163863  0.001551   19.190588   69.808518

In [106]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [107]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

- Devolvendo dados agregados sem índices de linha

In [108]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True)


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## _Método apply: separar-aplicar-combinar genérico_

In [109]:
# Função que seleciona as linhas com os maiores valores
# em uma coluna em particular
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=10)

,total_bill,tip,smoker,day,time,size,tip_pct
51,10.29,2.60,No,Sun,Dinner,2,0.252672
221,13.42,3.48,Yes,Fri,Lunch,2,0.259314
93,16.32,4.30,Yes,Fri,Dinner,2,0.263480
149,7.51,2.00,No,Thur,Lunch,2,0.266312
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [110]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [111]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [112]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [113]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

- Suprimindo as chaves de grupo

In [114]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


- Análise de quantis e de buckets

In [115]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-0.16, 1.434]
1     (-0.16, 1.434]
2    (-1.755, -0.16]
3    (-1.755, -0.16]
4     (-0.16, 1.434]
5    (-1.755, -0.16]
6     (-0.16, 1.434]
7    (-1.755, -0.16]
8    (-1.755, -0.16]
9     (-0.16, 1.434]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.356, -1.755] < (-1.755, -0.16] < (-0.16, 1.434] < (1.434, 3.029]]

In [116]:
def get_starts(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
    
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_starts).unstack()

,min,max,count,mean
data1,,,,
"(-3.356, -1.755]",-2.447927,1.926599,42.0,0.247134
"(-1.755, -0.16]",-4.389571,2.565348,400.0,-0.015501
"(-0.16, 1.434]",-4.128667,3.259676,485.0,-0.045288
"(1.434, 3.029]",-2.892312,2.925965,73.0,0.074114


In [117]:
# Devolve os números dos quantis
grouping = pd.qcut(frame.data1, 5, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_starts).unstack()

,min,max,count,mean
data1,,,,
0,-2.447927,2.543064,200.0,0.018298
1,-4.389571,2.323984,200.0,-0.006096
2,-4.128667,3.259676,200.0,-0.018835
3,-2.509292,2.579177,200.0,-0.019970
4,-2.892312,2.925965,200.0,-0.035274


- _Exemplo: Preenchando valores ausnetes com valores específicos de grupos_

In [118]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.379775
2         NaN
3    1.586124
4         NaN
5   -1.677686
dtype: float64

In [119]:
s.fillna(s.mean()) #preecnhe os dados ausentes com o valor da media

0    0.096071
1    0.379775
2    0.096071
3    1.586124
4    0.096071
5   -1.677686
dtype: float64

In [120]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon',
          'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.281759
New York     -0.928299
Vermont       1.833942
Florida       1.028074
Oregon       -0.309732
Nevada       -1.978242
California    1.685827
Idaho         0.395169
dtype: float64

In [121]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.281759
New York     -0.928299
Vermont            NaN
Florida       1.028074
Oregon       -0.309732
Nevada             NaN
California    1.685827
Idaho              NaN
dtype: float64

In [122]:
data.groupby(group_key).mean()

East   -0.060661
West    0.688047
dtype: float64

In [123]:
#preenchendo os valores NA usando as médias dos grupos
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.281759
      New York     -0.928299
      Vermont      -0.060661
      Florida       1.028074
West  Oregon       -0.309732
      Nevada        0.688047
      California    1.685827
      Idaho         0.688047
dtype: float64

In [124]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

East  Ohio         -0.281759
      New York     -0.928299
      Vermont       0.500000
      Florida       1.028074
West  Oregon       -0.309732
      Nevada       -1.000000
      California    1.685827
      Idaho        -1.000000
dtype: float64

- _Exemplo: Amostragem aleatória e permutação_

In [125]:
# Copas (Hearts), Espada (Spades), Paus (Clubs), Ouro (Diamonds)
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [126]:
# sorteio de uma mão de cinco cartas do baralho
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

2H    2
6C    6
9S    9
7H    7
4D    4
dtype: int64

In [127]:
get_suit = lambda card: card[-1] # ultima letra é o naipe
deck.groupby(get_suit).apply(draw, n=2)

C  2C    2
   AC    1
D  9D    9
   3D    3
H  3H    3
   7H    7
S  6S    6
   3S    3
dtype: int64

In [128]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

6C     6
JC    10
KD    10
2D     2
7H     7
5H     5
KS    10
7S     7
dtype: int64

- _Exemplo: média ponderada de grupos e correlação_

In [129]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                                'data': np.random.randn(8),
                                'weights': np.random.randn(8)})
df

,category,data,weights
0,a,0.566972,0.061182
1,a,-1.306374,1.487315
2,a,-0.096935,-0.793770
3,a,0.920648,-2.237853
4,b,0.255687,1.845071
5,b,-0.740212,-0.524858
6,b,1.197602,-1.108224
7,b,-1.469215,0.842245


In [135]:
# media ponderada dos grupos por category
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    2.623939
b   -1.616703
dtype: float64

In [137]:
close_px = pd.read_csv('exemplos/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [132]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [133]:
# calcula a correlação aos pares entre cada coluna e a coluna SPX
spx_coor = lambda x: x.corrwith(x['SPX'])

# calculando as mudaças percertuais em close_px usando pct_change
rest = close_px.pct_change().dropna()

# agrupamento das mudanças percentuais 
get_year = lambda x: x.year
by_year = rest.groupby(get_year)
by_year.apply(spx_coor)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [134]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

- _Exemplo: Regressão linear nos grupos_

In [138]:
import statsmodels.api as sm

# função regress executa uma regressão OLS em cada porção dos dados
def regress (data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

# execulta uma regressão linear anual de AAPL nos retornos de SPX
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## _Tabelas pivõs e tabulação cruzada_

In [141]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [142]:
tips.pivot_table(['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns='smoker')


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [144]:
tips.pivot_table(['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns='smoker', margins=True)
# margins = true adiciona All 

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [146]:
tips.pivot_table('tip_pct',
                 index=['time', 'smoker'],
                 columns='day', margins=True, aggfunc=len)
#aggfunc = len ou 'cout' leva a uma contagem dos grupos

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [150]:
#excluindo valores NA
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0, margins=True)

day                      Fri       Sat       Sun      Thur       All
time   size smoker                                                  
Dinner 1    No      0.000000  0.137931  0.000000  0.000000  0.137931
            Yes     0.000000  0.325733  0.000000  0.000000  0.325733
       2    No      0.139622  0.162705  0.168859  0.159744  0.164383
            Yes     0.171297  0.148668  0.207893  0.000000  0.167246
       3    No      0.000000  0.154661  0.152663  0.000000  0.153705
            Yes     0.000000  0.144995  0.152660  0.000000  0.148061
       4    No      0.000000  0.150096  0.148143  0.000000  0.148737
            Yes     0.117750  0.124515  0.193370  0.000000  0.139064
       5    No      0.000000  0.000000  0.206928  0.000000  0.206928
            Yes     0.000000  0.106572  0.065660  0.000000  0.086116
       6    No      0.000000  0.000000  0.103799  0.000000  0.103799
Lunch  1    No      0.000000  0.000000  0.000000  0.181728  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000  0.223776
       2    No      0.000000  0.000000  0.000000  0.166005  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843  0.165266
       3    No      0.187735  0.000000  0.000000  0.084246  0.118742
            Yes     0.000000  0.000000  0.000000  0.204952  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706  0.173706
All                 0.169913  0.153152  0.166897  0.161276  0.160803

- Tabulação cruzada: crosstab

In [153]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [157]:
# calculo das frequências dos grupos
pd.crosstab(index=data.Nationality, columns=data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [158]:
pd.crosstab(index=[tips.time, tips.day], columns=tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244